## How to run the experiments

Run the code blocs bellow in sequence. You can read the descriptions to understand it.


The dependencies can be found in https://github.com/eduardogc8/simple-qc

Before starting to run the experiments, change the variable ``path_wordembedding``, in the code block below, for the correct directory path. Make sure that the word embedding inside follow the template `wiki.multi.*.vec`.

In [1]:
import nltk
import numpy as np
import pandas as pd
from keras.preprocessing.sequence import pad_sequences
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import normalize

from benchmarking_methods import run_benchmark
from building_classifiers import lstm_default, svm_linear, random_forest, cnn
from download_word_embeddings import muse_embeddings_path, download_if_not_existing
from loading_data import load_embedding, load_uiuc

path_wordembedding = '/home/eduardo/word_embedding/'
download_if_not_existing()
from benchmarking_methods import run_benchmark_cv
from feature_creation import create_feature
from loading_data import load_disequa

Using TensorFlow backend.


### Extract features

The function *create_features* transform the questions in numerical vector to a classifier model.<br>It returns the output in the df_2 dataframe that is a parameter (*df_2.feature_type*, according to the *feature_type*).<br><br>
**feature_type:** type of feature. (bow, tfidf, embedding, embedding_sum, vocab_index, pos_index, pos_hotencode, ner_index, ner_hotencode)<br> 
**df:** the dataframe used to fit the transformers models (df.questions).<br>
**df_2:** dataframe wich the data will be transformed (df_2.questions).<br>
**embedding:** embedding model for word embedding features type.<br>
**max_features:** used in bag-of-words and TFIDF.


### Create classifier models

The models are created through functions that return them. These functions will be used to create a new model in each experiment. Therefore, an instance of a model is created by the benchmark function and not explicitly in a code block.


### UTILS

In [2]:
import warnings
warnings.filterwarnings("ignore")



#### Load UIUC dataset

#### Load DISEQuA dataset

## Benchmark UIUC - Normal

**Normal:** it uses the default fixed split of UIUC between train dataset (at last 5500 instances) and test dataset (500 instances). Therefore, it does not use cross-validation.

When the *run_benchmark* function is executed, it will save each result in the *save* path.

**model:** a dictionary with the classifier name and the function to create and return the model (not an instance of the model). <br> Example: *model = {'name': 'SVM', 'model': svm_linear}*<br>
**X:** all the training set.<br>
**y:** all the labels of the training set.<br>
**x_test:** test set.<br>
**y_test:** labels of the test set.<br>
**sizes_train:** sizes of training set. For each size, an experiment is executed.<br>
**runs:** number of time that each experiment is executed (used in models which has parameters with random values, like weights in an ANN).<br>
**save:** csv path where the results will be saved.<br>
**metric_average:** used in f1, recall and precision metrics<br>
**onehot:** one-hot model to transform labels.<br>
**out_dim:** the total of classes for ANN models.<br>
**epochs:** epochs for ANN models.<br>
**batch_size:** batch_size for ANN models.<br>
**vocabulary_size:** vocabulary size (used in CNN model).



## Benchmark UIUC and DISEQuA - Cross-validation

**Cross-validation:** instead of uses default fixed splits, it uses the all the dataset with cross-validation with 10 folds.

When the *run_benchmark* function is executed, it will save each result in the *save* path.

**model:** a dictionary with the classifier name and the function to create and return the model (not an instance of the model). <br> Example: *model = {'name': 'SVM', 'model': svm_linear}*<br>
**X:** Input features.<br>
**y:** Input labels.<br>
**sizes_train:** sizes of training set. For each size, an experiment is executed.<br>
**folds:** Amount of folds for cross-validations.<br>
**save:** csv path where the results will be saved.<br>
**metric_average:** used in f1, recall and precision metrics<br>
**onehot:** one-hot model to transform labels.<br>
**epochs:** epochs for ANN models.<br>
**batch_size:** batch_size for ANN models.<br>
**vocabulary_size:** vocabulary size (used in CNN model).



## Run UIUC Benchmark - Normal

Different classifier models are tested with different dependency levels of external linguistic resources (Low, Medium and High)

#### SVM + TF-IDF

In [ ]:
for language in ['en', 'es', 'pt']:
    print('\n\nLanguage: ', language)
    dataset_train, dataset_test = load_uiuc(language)
    create_feature('tfidf', dataset_train, dataset_train, max_features=2000)
    create_feature('tfidf', dataset_train, dataset_test, max_features=2000)
    
    model = {'name': 'svm', 'model': svm_linear}
    
    tfidf_train = np.array([list(r) for r in dataset_train['tfidf'].values])
    tfidf_test = np.array([list(r) for r in dataset_test['tfidf'].values])
    tfidf_train = normalize(tfidf_train, norm='max')
    tfidf_test = normalize(tfidf_test, norm='max')
    
    X_train = np.array([list(x) for x in dataset_train['tfidf'].values])
    X_test = np.array([list(x) for x in dataset_test['tfidf'].values])
    y_train = dataset_train['class'].values
    y_test = dataset_test['class'].values
    
    run_benchmark(model, X_train, y_train, X_test, y_test, sizes_train=[1000, 2000, 3000, 4000, 5500],
                  save='results/UIUC_svm_tfidf_' + language + '.csv', runs=1)

#### SVM + TF-IDF + WB

In [ ]:
for language in ['en', 'es', 'pt']:
    print('\n\nLanguage: ', language)
    embedding = load_embedding(path_wordembedding + 'wiki.multi.' + language + '.vec')
    dataset_train, dataset_test = load_uiuc(language)
    create_feature('tfidf', dataset_train, dataset_train, max_features=2000)
    create_feature('tfidf', dataset_train, dataset_test, max_features=2000)
    create_feature('embedding_sum', None, dataset_train, embedding)
    create_feature('embedding_sum', None, dataset_test, embedding)
    
    model = {'name': 'svm', 'model': svm_linear}
    
    tfidf_train = np.array([list(r) for r in dataset_train['tfidf'].values])
    tfidf_test = np.array([list(r) for r in dataset_test['tfidf'].values])
    tfidf_train = normalize(tfidf_train, norm='max')
    tfidf_test = normalize(tfidf_test, norm='max')
    
    embedding_train = np.array([list(r) for r in dataset_train['embedding_sum'].values])
    embedding_test = np.array([list(r) for r in dataset_test['embedding_sum'].values])
    embedding_train = normalize(embedding_train, norm='max')
    embedding_test = normalize(embedding_test, norm='max')
    
    X_train = np.array([list(x) + list(xx) for x, xx in zip(tfidf_train, embedding_train)])
    X_test = np.array([list(x) + list(xx) for x, xx in zip(tfidf_test, embedding_test)])
    y_train = dataset_train['class'].values
    y_test = dataset_test['class'].values
    
    run_benchmark(model, X_train, y_train, X_test, y_test, sizes_train=[1000, 2000, 3000, 4000, 5500], 
                  runs=1, save='results/UIUC_svm_cortes_' + language + '.csv')

#### SVM + TF-IDF + WB + POS + NER

In [ ]:
for language in ['en', 'es', 'pt']:
    print('\n\nLanguage: ', language)
    embedding = load_embedding(path_wordembedding + 'wiki.multi.' + language + '.vec')
    dataset_train, dataset_test = load_uiuc(language)
    create_feature('tfidf', dataset_train, dataset_train, max_features=2000)
    create_feature('tfidf', dataset_train, dataset_test, max_features=2000)
    create_feature('embedding_sum', dataset_train, dataset_train, embedding)
    create_feature('embedding_sum', dataset_train, dataset_test, embedding)
    create_feature('pos_hotencode', dataset_train, dataset_train)
    create_feature('pos_hotencode', dataset_train, dataset_test)
    create_feature('ner_hotencode', dataset_train, dataset_train)
    create_feature('ner_hotencode', dataset_train, dataset_test)
    model = {'name': 'svm', 'model': svm_linear}
    
    tfidf_train = np.array([list(r) for r in dataset_train['tfidf'].values])
    tfidf_test = np.array([list(r) for r in dataset_test['tfidf'].values])
    tfidf_train = normalize(tfidf_train, norm='max')
    tfidf_test = normalize(tfidf_test, norm='max')
    
    embedding_train = np.array([list(r) for r in dataset_train['embedding_sum'].values])
    embedding_test = np.array([list(r) for r in dataset_test['embedding_sum'].values])
    embedding_train = normalize(embedding_train, norm='max')
    embedding_test = normalize(embedding_test, norm='max')
    
    pos_train = np.array([list(r) for r in dataset_train['pos_hotencode'].values])
    pos_test = np.array([list(r) for r in dataset_test['pos_hotencode'].values])
    
    ner_train = np.array([list(r) for r in dataset_train['ner_hotencode'].values])
    ner_test = np.array([list(r) for r in dataset_test['ner_hotencode'].values])
    
    X_train = np.array([list(x) + list(xx) + list(xxx) + list(xxxx) for x, xx, xxx, xxxx in zip(tfidf_train, embedding_train, pos_train, ner_train)])
    X_test = np.array([list(x) + list(xx) + list(xxx) + list(xxxx) for x, xx, xxx, xxxx in zip(tfidf_test, embedding_test, pos_test, ner_test)])
    
    y_train = dataset_train['class'].values
    y_test = dataset_test['class'].values
    
    classes = list(dataset_train['class'].unique())
    y_train_ = [classes.index(c) for c in y_train]
    
    run_benchmark(model, X_train, y_train, X_test, y_test, sizes_train=[1000, 2000, 3000, 4000, 5500],
                  runs=1, save='results/UIUC_svm_high_' + language + '.csv')

#### BERT + CNN

In [ ]:
from typing import List
from flair_cnn_doc_embedding import DocumentCNNEmbeddings
from torch.utils.data import Dataset
import torch
from flair.data import Sentence, Corpus
from flair.embeddings import DocumentRNNEmbeddings, BertEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
import time
import datetime
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, confusion_matrix


def build_flair_sentences(text_label_tuples):
    sentences = [Sentence(text, labels=[label], use_tokenizer=True) for text,label in text_label_tuples]
    return [s for s in sentences if len(s.tokens) > 0]

def get_labels(sentences:List[Sentence]):
    return [[l.value for l in s.labels] for s in sentences]


def calc_metrics_with_sklearn(clf:TextClassifier,sentences:List[Sentence],train_size=0,
                              run=0,train_time=0,metric_average='macro',
                              classes=['ABBR', 'DESC', 'ENTY', 'HUM', 'LOC', 'NUM']):
    targets = get_labels(sentences)
    start_time = time.time()
    clf.predict(sentences)
    test_time = time.time() - start_time
    prediction = get_labels(sentences)
    p = prediction
    t = targets
    data = {'datetime': datetime.datetime.now(),
            'model': 'cnn_bert',
            'accuracy': accuracy_score(prediction, targets),
            'precision': precision_score(prediction, targets, average=metric_average),
            'recall': recall_score(prediction, targets, average=metric_average),
            'f1': f1_score(prediction, targets, average=metric_average),
            'mcc': matthews_corrcoef(prediction, targets),
            'confusion': confusion_matrix(prediction, targets, labels=classes),
            'run': run,
            'train_size': size_train,
            'execution_time': train_time,
            'test_time': test_time}
          
    #report = metrics.classification_report(y_true=targets, y_pred=prediction, digits=3, output_dict=True)
    return data


for language in ['en', 'es', 'pt']: # , 
    results = pd.DataFrame()
    
    save = 'results/UIUC_cnn_bert_'+language+'.csv'
    for size_train in [1000, 2000, 3000, 4000, 5500]:
        for run in range(2):
            dataset_train, dataset_test = load_uiuc(language)
            if size_train < 5500:
                dataset_train = dataset_train[:size_train]

            sentences_train:Dataset = build_flair_sentences([(text, label) for text, label in zip(dataset_train['question'], dataset_train['class'])])
            sentences_dev:Dataset = sentences_train
            sentences_test:Dataset = build_flair_sentences([(text, label) for text, label in zip(dataset_test['question'], dataset_test['class'])])

            corpus:Corpus = Corpus(sentences_train, sentences_dev, sentences_test)
            label_dict = corpus.make_label_dictionary()
            word_embeddings = [
                # WordEmbeddings('glove'),
                BertEmbeddings('bert-base-multilingual-cased', layers='-1')
            ]
            document_embeddings = DocumentCNNEmbeddings(word_embeddings,
                                                        dropout=0.0,
                                                        hidden_size=64,
                                                        )

            clf = TextClassifier(document_embeddings, label_dictionary=label_dict, multi_label=False)
            trainer = ModelTrainer(clf, corpus,torch.optim.RMSprop)
            base_path = 'flair_resources/qc_en_uiuc'
            start_time = time.time()
            trainer.train(base_path,
                          learning_rate=0.001,
                          mini_batch_size=32,
                          anneal_factor=0.5,
                          patience=2,
                          max_epochs=5)
            train_time = time.time() - start_time
            data = calc_metrics_with_sklearn(clf, sentences_test, train_size=size_train, train_time=train_time, run=run)
            results = results.append([data])
            results.to_csv(save)

2019-09-04 09:51:33,295 {'ABBR', 'ENTY', 'NUM', 'LOC', 'DESC', 'HUM'}
2019-09-04 09:51:33,296 The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.
2019-09-04 09:51:48,811 ----------------------------------------------------------------------------------------------------
2019-09-04 09:51:48,812 Evaluation method: MICRO_F1_SCORE
2019-09-04 09:51:49,419 ----------------------------------------------------------------------------------------------------
2019-09-04 09:51:50,650 epoch 1 - iter 0/32 - loss 2.12006998
2019-09-04 09:51:52,887 epoch 1 - iter 3/32 - loss 9.45129418
2019-09-04 09:51:55,205 epoch 1 - iter 6/32 - loss 9.43634268
2019-09-04 09:51:57,319 epoch 1 - iter 9/32 - loss 8.02415946
2019-09-04 09:51:59,325 epoch 1 - iter 12/32 - loss 6.66586506
2019-09-04 09:52:01,627 epoch 1 - iter 15/32 - loss 5.90735512
2019-09-04 09:52:03,891 epoch 1 - it

2019-09-04 09:57:47,423 epoch 1 - iter 6/32 - loss 9.96088454
2019-09-04 09:57:50,021 epoch 1 - iter 9/32 - loss 7.81378376
2019-09-04 09:57:52,419 epoch 1 - iter 12/32 - loss 6.58193128
2019-09-04 09:57:54,494 epoch 1 - iter 15/32 - loss 5.82463807
2019-09-04 09:57:56,293 epoch 1 - iter 18/32 - loss 5.18112130
2019-09-04 09:57:59,122 epoch 1 - iter 21/32 - loss 4.75785893
2019-09-04 09:58:01,578 epoch 1 - iter 24/32 - loss 4.40232936
2019-09-04 09:58:04,080 epoch 1 - iter 27/32 - loss 4.11624760
2019-09-04 09:58:06,003 epoch 1 - iter 30/32 - loss 3.87416960
2019-09-04 09:58:06,373 ----------------------------------------------------------------------------------------------------
2019-09-04 09:58:06,374 EPOCH 1 done: loss 3.7913 - lr 0.0010 - bad epochs 0
2019-09-04 09:58:29,019 DEV : loss 1.304010033607483 - score 0.461
2019-09-04 09:58:37,414 TEST : loss 1.2612091302871704 - score 0.42
2019-09-04 09:58:43,575 --------------------------------------------------------------------------

2019-09-04 10:05:01,278 DEV : loss 0.6842145919799805 - score 0.7555
2019-09-04 10:05:10,140 TEST : loss 0.7059192657470703 - score 0.722
2019-09-04 10:05:16,036 ----------------------------------------------------------------------------------------------------
2019-09-04 10:05:17,837 epoch 2 - iter 0/63 - loss 0.46725100
2019-09-04 10:05:22,771 epoch 2 - iter 6/63 - loss 0.59196515
2019-09-04 10:05:27,084 epoch 2 - iter 12/63 - loss 0.56492399
2019-09-04 10:05:31,695 epoch 2 - iter 18/63 - loss 0.53312009
2019-09-04 10:05:36,421 epoch 2 - iter 24/63 - loss 0.52741346
2019-09-04 10:05:41,050 epoch 2 - iter 30/63 - loss 0.51177767
2019-09-04 10:05:45,661 epoch 2 - iter 36/63 - loss 0.53441886
2019-09-04 10:05:49,890 epoch 2 - iter 42/63 - loss 0.51797272
2019-09-04 10:05:54,397 epoch 2 - iter 48/63 - loss 0.51177592
2019-09-04 10:05:58,554 epoch 2 - iter 54/63 - loss 0.50460576
2019-09-04 10:06:03,085 epoch 2 - iter 60/63 - loss 0.50746983
2019-09-04 10:06:04,589 ----------------------

2019-09-04 10:15:57,702 epoch 2 - iter 42/63 - loss 0.58137968
2019-09-04 10:16:03,076 epoch 2 - iter 48/63 - loss 0.57741890
2019-09-04 10:16:07,971 epoch 2 - iter 54/63 - loss 0.56740353
2019-09-04 10:16:13,173 epoch 2 - iter 60/63 - loss 0.56195580
2019-09-04 10:16:14,368 ----------------------------------------------------------------------------------------------------
2019-09-04 10:16:14,370 EPOCH 2 done: loss 0.5730 - lr 0.0010 - bad epochs 0
2019-09-04 10:16:59,843 DEV : loss 0.5737391710281372 - score 0.777
2019-09-04 10:17:08,867 TEST : loss 0.6028417944908142 - score 0.75
2019-09-04 10:17:08,869 ----------------------------------------------------------------------------------------------------
2019-09-04 10:17:10,342 epoch 3 - iter 0/63 - loss 0.49918604
2019-09-04 10:17:15,782 epoch 3 - iter 6/63 - loss 0.40472653
2019-09-04 10:17:21,103 epoch 3 - iter 12/63 - loss 0.38063633
2019-09-04 10:17:25,495 epoch 3 - iter 18/63 - loss 0.36335808
2019-09-04 10:17:30,277 epoch 3 - i

2019-09-04 10:29:01,539 epoch 3 - iter 0/94 - loss 0.11116195
2019-09-04 10:29:10,353 epoch 3 - iter 9/94 - loss 0.25565144
2019-09-04 10:29:17,252 epoch 3 - iter 18/94 - loss 0.27018153
2019-09-04 10:29:24,670 epoch 3 - iter 27/94 - loss 0.26828345
2019-09-04 10:29:32,158 epoch 3 - iter 36/94 - loss 0.28704919
2019-09-04 10:29:39,693 epoch 3 - iter 45/94 - loss 0.29236101
2019-09-04 10:29:46,898 epoch 3 - iter 54/94 - loss 0.28889667
2019-09-04 10:29:54,633 epoch 3 - iter 63/94 - loss 0.29517570
2019-09-04 10:30:02,800 epoch 3 - iter 72/94 - loss 0.28678324
2019-09-04 10:30:10,170 epoch 3 - iter 81/94 - loss 0.28909662
2019-09-04 10:30:17,922 epoch 3 - iter 90/94 - loss 0.28671391
2019-09-04 10:30:20,724 ----------------------------------------------------------------------------------------------------
2019-09-04 10:30:20,725 EPOCH 3 done: loss 0.2844 - lr 0.0010 - bad epochs 0
2019-09-04 10:31:39,816 DEV : loss 0.19301404058933258 - score 0.935
2019-09-04 10:31:49,411 TEST : loss 0.

2019-09-04 10:44:33,322 ----------------------------------------------------------------------------------------------------
2019-09-04 10:44:33,323 EPOCH 3 done: loss 0.2794 - lr 0.0010 - bad epochs 0
2019-09-04 10:45:42,356 DEV : loss 0.16736845672130585 - score 0.943
2019-09-04 10:45:50,778 TEST : loss 0.2419501543045044 - score 0.926
2019-09-04 10:45:59,874 ----------------------------------------------------------------------------------------------------
2019-09-04 10:46:01,306 epoch 4 - iter 0/94 - loss 0.10214311
2019-09-04 10:46:08,063 epoch 4 - iter 9/94 - loss 0.21456417
2019-09-04 10:46:14,745 epoch 4 - iter 18/94 - loss 0.20006843
2019-09-04 10:46:22,112 epoch 4 - iter 27/94 - loss 0.19525889
2019-09-04 10:46:28,757 epoch 4 - iter 36/94 - loss 0.19972057
2019-09-04 10:46:35,891 epoch 4 - iter 45/94 - loss 0.19810332
2019-09-04 10:46:42,889 epoch 4 - iter 54/94 - loss 0.19346358
2019-09-04 10:46:49,650 epoch 4 - iter 63/94 - loss 0.19573942
2019-09-04 10:46:56,652 epoch 4 -

2019-09-04 11:03:06,652 epoch 4 - iter 36/125 - loss 0.17444576
2019-09-04 11:03:15,947 epoch 4 - iter 48/125 - loss 0.16348123
2019-09-04 11:03:24,055 epoch 4 - iter 60/125 - loss 0.16429276
2019-09-04 11:03:32,684 epoch 4 - iter 72/125 - loss 0.16975183
2019-09-04 11:03:41,866 epoch 4 - iter 84/125 - loss 0.17589451
2019-09-04 11:03:50,725 epoch 4 - iter 96/125 - loss 0.17748731
2019-09-04 11:03:59,133 epoch 4 - iter 108/125 - loss 0.18271939
2019-09-04 11:04:08,674 epoch 4 - iter 120/125 - loss 0.18181382
2019-09-04 11:04:12,212 ----------------------------------------------------------------------------------------------------
2019-09-04 11:04:12,213 EPOCH 4 done: loss 0.1821 - lr 0.0010 - bad epochs 0
2019-09-04 11:05:42,197 DEV : loss 0.16616423428058624 - score 0.937
2019-09-04 11:05:50,742 TEST : loss 0.2245163768529892 - score 0.908
2019-09-04 11:05:50,743 ----------------------------------------------------------------------------------------------------
2019-09-04 11:05:52,1

2019-09-04 11:21:18,116 EPOCH 4 done: loss 0.1878 - lr 0.0010 - bad epochs 0
2019-09-04 11:22:48,597 DEV : loss 0.13149404525756836 - score 0.9537
2019-09-04 11:22:56,769 TEST : loss 0.23505082726478577 - score 0.922
2019-09-04 11:23:03,304 ----------------------------------------------------------------------------------------------------
2019-09-04 11:23:04,899 epoch 5 - iter 0/125 - loss 0.04910938
2019-09-04 11:23:14,548 epoch 5 - iter 12/125 - loss 0.11135908
2019-09-04 11:23:22,991 epoch 5 - iter 24/125 - loss 0.10982776
2019-09-04 11:23:33,005 epoch 5 - iter 36/125 - loss 0.13260033
2019-09-04 11:23:42,665 epoch 5 - iter 48/125 - loss 0.14098764
2019-09-04 11:23:51,895 epoch 5 - iter 60/125 - loss 0.14244775
2019-09-04 11:24:01,916 epoch 5 - iter 72/125 - loss 0.14217578
2019-09-04 11:24:12,886 epoch 5 - iter 84/125 - loss 0.14565166
2019-09-04 11:24:23,589 epoch 5 - iter 96/125 - loss 0.14335153
2019-09-04 11:24:34,239 epoch 5 - iter 108/125 - loss 0.15501967
2019-09-04 11:24:4

2019-09-04 11:47:12,235 epoch 5 - iter 68/171 - loss 0.09443624
2019-09-04 11:47:28,391 epoch 5 - iter 85/171 - loss 0.08765763
2019-09-04 11:47:41,173 epoch 5 - iter 102/171 - loss 0.08705867
2019-09-04 11:47:55,097 epoch 5 - iter 119/171 - loss 0.08423933
2019-09-04 11:48:08,709 epoch 5 - iter 136/171 - loss 0.08248052
2019-09-04 11:48:22,598 epoch 5 - iter 153/171 - loss 0.08123493
2019-09-04 11:48:35,322 epoch 5 - iter 170/171 - loss 0.08316526
2019-09-04 11:48:35,668 ----------------------------------------------------------------------------------------------------
2019-09-04 11:48:35,670 EPOCH 5 done: loss 0.0832 - lr 0.0005 - bad epochs 0
2019-09-04 11:51:08,459 DEV : loss 0.05454140529036522 - score 0.9872
2019-09-04 11:51:17,698 TEST : loss 0.14391571283340454 - score 0.95
2019-09-04 11:51:30,161 ----------------------------------------------------------------------------------------------------
2019-09-04 11:51:30,162 Testing using best model ...
2019-09-04 11:51:30,163 load

## Run UIUC Benchmark - Cross-validation

Different classifier models are tested with different dependency levels of external linguistic resources (Low, Medium and High)

#### SVM + TF-IDF

In [ ]:
for language in ['en', 'es', 'pt']:
    print('\n\nLanguage: ', language)
    dataset_train, dataset_test = load_uiuc(language)
    dataset = pd.concat([dataset_train, dataset_test])
    create_feature('tfidf', dataset, dataset, max_features=2000)
    
    model = {'name': 'svm', 'model': svm_linear}
    
    tfidf = np.array([list(r) for r in dataset['tfidf'].values])
    tfidf = normalize(tfidf, norm='max')
    
    X = np.array([list(x) for x in dataset['tfidf'].values])
    y = dataset['class'].values
    
    
    # run_benchmark_cv(model, X, y, [50, 100] + list(range(500, 5501, 500)),
    run_benchmark_cv(model, X, y, [1000, 2000, 3000, 4000, 5500],
                     save='results/UIUC_cv_svm_tfidf_' + language + '.csv')

#### SVM + TF-IDF + WB

In [ ]:
for language in ['en', 'es', 'pt']:
    print('\n\nLanguage: ', language)
    embedding = load_embedding(path_wordembedding + 'wiki.multi.' + language + '.vec')
    dataset_train, dataset_test = load_uiuc(language)
    dataset = pd.concat([dataset_train, dataset_test])
    create_feature('tfidf', dataset, dataset, max_features=2000)
    create_feature('embedding_sum', None, dataset, embedding)
    
    model = {'name': 'svm', 'model': svm_linear}
    
    tfidf = np.array([list(r) for r in dataset['tfidf'].values])
    tfidf = normalize(tfidf, norm='max')
    
    embedding = np.array([list(r) for r in dataset['embedding_sum'].values])
    embedding = normalize(embedding, norm='max')
    
    X = np.array([list(x) + list(xx) for x, xx in zip(tfidf, embedding)])
    y = dataset['class'].values
    
    # run_benchmark_cv(model, X, y, [50, 100] + list(range(500, 5501, 500)),
    run_benchmark_cv(model, X, y, [1000, 2000, 3000, 4000, 5500],
                     save='results/UIUC_cv_svm_cortes_' + language + '.csv')

#### SVM + TF-IDF + WB + POS + NER

In [ ]:
for language in ['en', 'es', 'pt']:
    print('\n\nLanguage: ', language)
    embedding = load_embedding(path_wordembedding + 'wiki.multi.' + language + '.vec')
    dataset_train, dataset_test = load_uiuc(language)
    dataset = pd.concat([dataset_train, dataset_test])
    create_feature('tfidf', dataset, dataset, max_features=2000)
    create_feature('embedding_sum', dataset, dataset, embedding)
    create_feature('pos_hotencode', dataset, dataset)
    create_feature('ner_hotencode', dataset, dataset)
    model = {'name': 'svm', 'model': svm_linear}
    
    tfidf = np.array([list(r) for r in dataset['tfidf'].values])
    tfidf = normalize(tfidf, norm='max')
    
    embedding = np.array([list(r) for r in dataset['embedding_sum'].values])
    embedding = normalize(embedding, norm='max')
    
    pos = np.array([list(r) for r in dataset['pos_hotencode'].values])
    
    ner = np.array([list(r) for r in dataset['ner_hotencode'].values])
    
    X = np.array([list(x) + list(xx) + list(xxx) + list(xxxx) for x, xx, xxx, xxxx in zip(tfidf, embedding, pos, ner)])
    
    y = dataset['class'].values
    
    # run_benchmark_cv(model, X, y, [50, 100] + list(range(500, 5501, 500)),
    run_benchmark_cv(model, X, y, [1000, 2000, 3000, 4000, 5500],
                     save='results/UIUC_cv_svm_high_' + language + '.csv')

#### BERT + CNN - Cross validation

In [6]:
from typing import List
from flair_cnn_doc_embedding import DocumentCNNEmbeddings
from torch.utils.data import Dataset
import torch
from flair.data import Sentence, Corpus
from flair.embeddings import DocumentRNNEmbeddings, BertEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
import time
import datetime
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, confusion_matrix


def build_flair_sentences(text_label_tuples):
    sentences = [Sentence(text, labels=[label], use_tokenizer=True) for text,label in text_label_tuples]
    return [s for s in sentences if len(s.tokens) > 0]

def get_labels(sentences:List[Sentence]):
    return [[l.value for l in s.labels] for s in sentences]

def calc_metrics_with_sklearn(clf:TextClassifier,sentences:List[Sentence],train_size=0,
                              fold=0,train_time=0,metric_average='macro',
                              classes=['ABBR', 'DESC', 'ENTY', 'HUM', 'LOC', 'NUM']):
    targets = get_labels(sentences)
    start_time = time.time()
    clf.predict(sentences)
    test_time = time.time() - start_time
    prediction = get_labels(sentences)
    p = prediction
    t = targets
    data = {'datetime': datetime.datetime.now(),
            'model': 'cnn_bert',
            'accuracy': accuracy_score(prediction, targets),
            'precision': precision_score(prediction, targets, average=metric_average),
            'recall': recall_score(prediction, targets, average=metric_average),
            'f1': f1_score(prediction, targets, average=metric_average),
            'mcc': matthews_corrcoef(prediction, targets),
            'confusion': confusion_matrix(prediction, targets, labels=classes),
            'fold': fold,
            'train_size': size_train,
            'execution_time': train_time,
            'test_time': test_time}

    #report = metrics.classification_report(y_true=targets, y_pred=prediction, digits=3, output_dict=True)
    return data




for language in ['es',]: # ,  'es', 'pt'
    print(f"########## {language} ##########")
    results = pd.DataFrame()
    dataset_train, dataset_test = load_uiuc(language)
    dataset = pd.concat([dataset_train, dataset_test])
    save = 'results/UIUC_cv_cnn_bert_'+language+'_1000_2000.csv'
    for size_train in [1000, 2000]: # 
        print(f"##### {size_train} #####")
        
        word_embeddings = [BertEmbeddings('bert-base-multilingual-cased', layers='-1')]
        document_embeddings = DocumentCNNEmbeddings(word_embeddings, dropout=0.0, hidden_size=64)
        
        size_test = len(dataset) - size_train
        rs = StratifiedShuffleSplit(n_splits=10, train_size=size_train, test_size=size_test, random_state=1)
        fold = 0
        for train_indexs, test_indexs in rs.split(dataset, dataset['class']):
            fold += 1
            print(f"## {fold} ##")
            df_train = dataset.iloc[train_indexs]
            df_test = dataset.iloc[test_indexs]
            
            x_train:Dataset = build_flair_sentences([(text, label) for text, label in zip(df_train['question'], df_train['class'])])
            x_dev:Dataset = x_train
            x_test:Dataset = build_flair_sentences([(text, label) for text, label in zip(df_test['question'], df_test['class'])])
            
            corpus = Corpus(x_train, x_dev, x_test)
            label_dict = corpus.make_label_dictionary()

            clf = TextClassifier(document_embeddings, label_dictionary=label_dict, multi_label=False)
            trainer = ModelTrainer(clf, corpus,torch.optim.RMSprop)
            base_path = 'flair_resources/qc_'+language+'_uiuc'
            start_time = time.time()
            trainer.train(base_path,
                          learning_rate=0.001,
                          mini_batch_size=32,
                          anneal_factor=0.5,
                          max_epochs=4,
                          patience=2,
                          )
            train_time = time.time() - start_time
            data = calc_metrics_with_sklearn(clf, x_test, train_size=size_train, train_time=train_time, fold=fold)
            results = results.append([data])
            results.to_csv(save)

########## es ##########
##### 1000 #####
2019-08-30 00:15:56,544 The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.
## 1 ##
2019-08-30 00:16:44,314 {'ENTY', 'NUM', 'ABBR', 'HUM', 'DESC', 'LOC'}
2019-08-30 00:16:44,526 ----------------------------------------------------------------------------------------------------
2019-08-30 00:16:44,527 Evaluation method: MICRO_F1_SCORE
2019-08-30 00:16:45,244 ----------------------------------------------------------------------------------------------------
2019-08-30 00:16:47,386 epoch 1 - iter 0/32 - loss 1.77986097
2019-08-30 00:16:50,576 epoch 1 - iter 3/32 - loss 8.04128671
2019-08-30 00:16:53,477 epoch 1 - iter 6/32 - loss 6.44657620
2019-08-30 00:16:56,890 epoch 1 - iter 9/32 - loss 5.41995763
2019-08-30 00:16:59,525 epoch 1 - iter 12/32 - loss 4.59803794
2019-08-30 00:17:02,758 epoch 1 - iter 15/32 - lo

2019-08-30 00:41:46,920 epoch 2 - iter 12/32 - loss 1.03961400
2019-08-30 00:41:49,447 epoch 2 - iter 15/32 - loss 1.00217039
2019-08-30 00:41:52,242 epoch 2 - iter 18/32 - loss 0.99930125
2019-08-30 00:41:55,333 epoch 2 - iter 21/32 - loss 1.03038987
2019-08-30 00:41:58,089 epoch 2 - iter 24/32 - loss 1.02364323
2019-08-30 00:42:01,085 epoch 2 - iter 27/32 - loss 0.98668079
2019-08-30 00:42:04,376 epoch 2 - iter 30/32 - loss 0.98622822
2019-08-30 00:42:06,090 ----------------------------------------------------------------------------------------------------
2019-08-30 00:42:06,092 EPOCH 2 done: loss 0.9702 - lr 0.0010 - bad epochs 0
2019-08-30 00:42:34,942 DEV : loss 0.6005484461784363 - score 0.776
2019-08-30 00:45:11,661 TEST : loss 0.9332205653190613 - score 0.672
2019-08-30 00:45:18,071 ----------------------------------------------------------------------------------------------------
2019-08-30 00:45:19,903 epoch 3 - iter 0/32 - loss 0.68040985
2019-08-30 00:45:22,861 epoch 3 -

2019-08-30 01:14:09,416 epoch 4 - iter 0/32 - loss 0.29436827
2019-08-30 01:14:12,317 epoch 4 - iter 3/32 - loss 0.36847810
2019-08-30 01:14:16,698 epoch 4 - iter 6/32 - loss 0.38930078
2019-08-30 01:14:19,076 epoch 4 - iter 9/32 - loss 0.39506946
2019-08-30 01:14:22,146 epoch 4 - iter 12/32 - loss 0.42571741
2019-08-30 01:14:24,620 epoch 4 - iter 15/32 - loss 0.43147716
2019-08-30 01:14:26,987 epoch 4 - iter 18/32 - loss 0.45169947
2019-08-30 01:14:29,940 epoch 4 - iter 21/32 - loss 0.46752783
2019-08-30 01:14:32,542 epoch 4 - iter 24/32 - loss 0.48703005
2019-08-30 01:14:35,146 epoch 4 - iter 27/32 - loss 0.48262915
2019-08-30 01:14:37,769 epoch 4 - iter 30/32 - loss 0.48836879
2019-08-30 01:14:51,805 ----------------------------------------------------------------------------------------------------
2019-08-30 01:14:51,992 EPOCH 4 done: loss 0.4909 - lr 0.0010 - bad epochs 0
2019-08-30 01:15:19,717 DEV : loss 0.3683728873729706 - score 0.866
2019-08-30 01:17:58,523 TEST : loss 0.722

2019-08-30 01:42:18,577 ----------------------------------------------------------------------------------------------------
## 5 ##
2019-08-30 01:44:56,002 {'ENTY', 'NUM', 'ABBR', 'HUM', 'DESC', 'LOC'}
2019-08-30 01:44:56,019 ----------------------------------------------------------------------------------------------------
2019-08-30 01:44:56,020 Evaluation method: MICRO_F1_SCORE
2019-08-30 01:44:56,674 ----------------------------------------------------------------------------------------------------
2019-08-30 01:44:59,045 epoch 1 - iter 0/32 - loss 8.46961308
2019-08-30 01:45:01,958 epoch 1 - iter 3/32 - loss 4.47838008
2019-08-30 01:45:05,070 epoch 1 - iter 6/32 - loss 3.46463159
2019-08-30 01:45:08,097 epoch 1 - iter 9/32 - loss 2.94123491
2019-08-30 01:45:10,924 epoch 1 - iter 12/32 - loss 2.61017027
2019-08-30 01:45:13,875 epoch 1 - iter 15/32 - loss 2.43157306
2019-08-30 01:45:17,212 epoch 1 - iter 18/32 - loss 2.27864429
2019-08-30 01:45:20,545 epoch 1 - iter 21/32 - loss 

2019-08-30 02:13:38,011 epoch 2 - iter 18/32 - loss 0.73612964
2019-08-30 02:13:40,875 epoch 2 - iter 21/32 - loss 0.74855922
2019-08-30 02:13:43,803 epoch 2 - iter 24/32 - loss 0.74200044
2019-08-30 02:13:46,585 epoch 2 - iter 27/32 - loss 0.74170662
2019-08-30 02:13:49,176 epoch 2 - iter 30/32 - loss 0.76656455
2019-08-30 02:13:50,114 ----------------------------------------------------------------------------------------------------
2019-08-30 02:13:50,115 EPOCH 2 done: loss 0.7595 - lr 0.0010 - bad epochs 0
2019-08-30 02:14:17,572 DEV : loss 0.642583966255188 - score 0.772
2019-08-30 02:16:55,673 TEST : loss 0.7775426506996155 - score 0.7306
2019-08-30 02:17:03,503 ----------------------------------------------------------------------------------------------------
2019-08-30 02:17:05,275 epoch 3 - iter 0/32 - loss 0.71338159
2019-08-30 02:17:08,624 epoch 3 - iter 3/32 - loss 0.63500477
2019-08-30 02:17:11,665 epoch 3 - iter 6/32 - loss 0.59191082
2019-08-30 02:17:14,069 epoch 3 - i

...

2019-08-30 06:59:13,901 epoch 4 - iter 12/63 - loss 0.41851586
2019-08-30 06:59:20,802 epoch 4 - iter 18/63 - loss 0.45900025
2019-08-30 06:59:26,353 epoch 4 - iter 24/63 - loss 0.43404794
2019-08-30 06:59:31,665 epoch 4 - iter 30/63 - loss 0.43845882
2019-08-30 06:59:37,607 epoch 4 - iter 36/63 - loss 0.43892490
2019-08-30 06:59:44,319 epoch 4 - iter 42/63 - loss 0.43396895
2019-08-30 07:01:28,265 epoch 4 - iter 48/63 - loss 0.43914601
2019-08-30 07:01:42,805 epoch 4 - iter 54/63 - loss 0.42909163
2019-08-30 07:02:40,623 epoch 4 - iter 60/63 - loss 0.44225843
2019-08-30 07:03:22,029 ----------------------------------------------------------------------------------------------------
2019-08-30 07:03:22,573 EPOCH 4 done: loss 0.4414 - lr 0.0010 - bad epochs 0
2019-08-30 07:04:34,845 DEV : loss 0.3516794741153717 - score 0.888
2019-08-30 07:07:01,847 TEST : loss 0.6665229201316833 - score 0.7995
2019-08-30 07:07:16,327 ---------------------------------------------------------------------

2019-08-30 07:45:45,857 ----------------------------------------------------------------------------------------------------
## 7 ##
2019-08-30 07:48:54,339 {'ENTY', 'NUM', 'ABBR', 'HUM', 'DESC', 'LOC'}
2019-08-30 07:48:54,388 ----------------------------------------------------------------------------------------------------
2019-08-30 07:48:54,390 Evaluation method: MICRO_F1_SCORE
2019-08-30 07:48:55,311 ----------------------------------------------------------------------------------------------------
2019-08-30 07:48:57,344 epoch 1 - iter 0/63 - loss 5.33133459
2019-08-30 07:49:03,858 epoch 1 - iter 6/63 - loss 3.62429563
2019-08-30 07:49:52,111 epoch 1 - iter 12/63 - loss 2.56970778
2019-08-30 07:50:14,790 epoch 1 - iter 18/63 - loss 2.23636854
2019-08-30 07:52:26,468 epoch 1 - iter 24/63 - loss 2.02165040
2019-08-30 07:52:54,131 epoch 1 - iter 30/63 - loss 1.85336732
2019-08-30 07:53:17,209 epoch 1 - iter 36/63 - loss 1.75582739
2019-08-30 07:53:38,659 epoch 1 - iter 42/63 - los

2019-08-30 08:45:25,182 epoch 2 - iter 36/63 - loss 1.32795631
2019-08-30 08:45:30,876 epoch 2 - iter 42/63 - loss 1.31881424
2019-08-30 08:45:57,924 epoch 2 - iter 48/63 - loss 1.30883798
2019-08-30 08:48:18,536 epoch 2 - iter 54/63 - loss 1.29632945
2019-08-30 08:48:28,273 epoch 2 - iter 60/63 - loss 1.29257091
2019-08-30 08:48:55,249 ----------------------------------------------------------------------------------------------------
2019-08-30 08:48:55,425 EPOCH 2 done: loss 1.2938 - lr 0.0010 - bad epochs 0
2019-08-30 08:50:17,722 DEV : loss 1.2849305868148804 - score 0.416
2019-08-30 08:53:50,785 TEST : loss 1.346233606338501 - score 0.4097
2019-08-30 08:54:02,082 ----------------------------------------------------------------------------------------------------
2019-08-30 08:54:04,375 epoch 3 - iter 0/63 - loss 1.16440558
2019-08-30 08:54:11,128 epoch 3 - iter 6/63 - loss 1.36838572
2019-08-30 08:54:17,907 epoch 3 - iter 12/63 - loss 1.32065621
2019-08-30 08:54:24,089 epoch 3 - 

2019-08-30 09:30:12,055 epoch 4 - iter 12/63 - loss 0.49906795
2019-08-30 09:30:17,569 epoch 4 - iter 18/63 - loss 0.50968976
2019-08-30 09:30:23,347 epoch 4 - iter 24/63 - loss 0.47567989
2019-08-30 09:30:29,608 epoch 4 - iter 30/63 - loss 0.49497867
2019-08-30 09:30:35,706 epoch 4 - iter 36/63 - loss 0.47769011
2019-08-30 09:30:40,469 epoch 4 - iter 42/63 - loss 0.48527218
2019-08-30 09:30:45,918 epoch 4 - iter 48/63 - loss 0.47183558
2019-08-30 09:30:51,205 epoch 4 - iter 54/63 - loss 0.47279594
2019-08-30 09:30:56,249 epoch 4 - iter 60/63 - loss 0.47369679
2019-08-30 09:30:58,388 ----------------------------------------------------------------------------------------------------
2019-08-30 09:30:58,389 EPOCH 4 done: loss 0.4740 - lr 0.0010 - bad epochs 0
2019-08-30 09:31:51,645 DEV : loss 0.5004297494888306 - score 0.8225
2019-08-30 09:34:03,928 TEST : loss 0.6718747615814209 - score 0.7799
2019-08-30 09:34:17,664 --------------------------------------------------------------------

2019-08-30 10:07:47,468 ----------------------------------------------------------------------------------------------------


## Run DISEQuA Benchmark - Cross-validation

Different classifier models are tested with different dependency levels of external linguistic resources (Low, Medium and High)

#### BERT + CNN - Cross validation

In [3]:
from typing import List
from flair_cnn_doc_embedding import DocumentCNNEmbeddings
from torch.utils.data import Dataset
import torch
from flair.data import Sentence, Corpus
from flair.embeddings import DocumentRNNEmbeddings, BertEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
import time
import datetime
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, confusion_matrix


def build_flair_sentences(text_label_tuples):
    sentences = [Sentence(text, labels=[label], use_tokenizer=True) for text,label in text_label_tuples]
    return [s for s in sentences if len(s.tokens) > 0]

def get_labels(sentences:List[Sentence]):
    return [[l.value for l in s.labels] for s in sentences]

def calc_metrics_with_sklearn(clf:TextClassifier,sentences:List[Sentence],train_size=0,
                              fold=0,train_time=0,metric_average='macro',
                              classes=['DATE', 'ORGANIZATION', 'PERSON', 'MEASURE', 'OTHER', 'OBJECT', 'LOCATION']):
    targets = get_labels(sentences)
    start_time = time.time()
    clf.predict(sentences)
    test_time = time.time() - start_time
    prediction = get_labels(sentences)
    p = prediction
    t = targets
    data = {'datetime': datetime.datetime.now(),
            'model': 'cnn_bert',
            'accuracy': accuracy_score(prediction, targets),
            'precision': precision_score(prediction, targets, average=metric_average),
            'recall': recall_score(prediction, targets, average=metric_average),
            'f1': f1_score(prediction, targets, average=metric_average),
            'mcc': matthews_corrcoef(prediction, targets),
            'confusion': confusion_matrix(prediction, targets, labels=classes),
            'fold': fold,
            'train_size': size_train,
            'execution_time': train_time,
            'test_time': test_time}

    #report = metrics.classification_report(y_true=targets, y_pred=prediction, digits=3, output_dict=True)
    return data




for language in ['nl', 'es', 'en', 'it']:  
    print(f"########## {language} ##########")
    results = pd.DataFrame()
    dataset = load_disequa(language)
    save = 'results/DISEQuA_cnn_bert_'+language+'.csv'
    for size_train in [200, 300, 400]: # 100
        print(f"##### {size_train} #####")
        
        word_embeddings = [BertEmbeddings('bert-base-multilingual-cased', layers='-1')]
        document_embeddings = DocumentCNNEmbeddings(word_embeddings, dropout=0.0, hidden_size=64)
        
        size_test = len(dataset) - size_train
        rs = StratifiedShuffleSplit(n_splits=10, train_size=size_train, test_size=size_test, random_state=1)
        fold = 0
        for train_indexs, test_indexs in rs.split(dataset, dataset['class']):
            fold += 1
            print(f"## {fold} ##")
            df_train = dataset.iloc[train_indexs]
            df_test = dataset.iloc[test_indexs]
            
            x_train:Dataset = build_flair_sentences([(text, label) for text, label in zip(df_train['question'], df_train['class'])])
            x_dev:Dataset = x_train
            x_test:Dataset = build_flair_sentences([(text, label) for text, label in zip(df_test['question'], df_test['class'])])
            
            corpus = Corpus(x_train, x_dev, x_test)
            label_dict = corpus.make_label_dictionary()

            clf = TextClassifier(document_embeddings, label_dictionary=label_dict, multi_label=False)
            trainer = ModelTrainer(clf, corpus,torch.optim.RMSprop)
            base_path = 'flair_resources/qc_'+language+'_disequa'
            start_time = time.time()
            trainer.train(base_path,
                          learning_rate=0.001,
                          mini_batch_size=32,
                          anneal_factor=0.5,
                          max_epochs=10,
                          patience=2,
                          )
            train_time = time.time() - start_time
            data = calc_metrics_with_sklearn(clf, x_test, train_size=size_train, train_time=train_time, fold=fold)
            results = results.append([data])
            results.to_csv(save)

########## nl ##########
##### 200 #####
2019-09-03 14:18:18,995 The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.
## 1 ##
2019-09-03 14:18:28,030 {'ORGANIZATION', 'OTHER', 'PERSON', 'MEASURE', 'LOCATION', 'DATE', 'OBJECT'}
2019-09-03 14:18:28,032 ----------------------------------------------------------------------------------------------------
2019-09-03 14:18:28,033 Evaluation method: MICRO_F1_SCORE
2019-09-03 14:18:28,197 ----------------------------------------------------------------------------------------------------
2019-09-03 14:18:28,989 epoch 1 - iter 0/7 - loss 2.15622735
2019-09-03 14:18:29,698 epoch 1 - iter 1/7 - loss 4.06451237
2019-09-03 14:18:30,438 epoch 1 - iter 2/7 - loss 9.02630623
2019-09-03 14:18:31,135 epoch 1 - iter 3/7 - loss 8.80788380
2019-09-03 14:18:31,841 epoch 1 - iter 4/7 - loss 8.60204778
2019-09-03 14:18:32,983 e

2019-09-03 14:21:29,672 ----------------------------------------------------------------------------------------------------
2019-09-03 14:21:29,673 EPOCH 9 done: loss 0.0161 - lr 0.0010 - bad epochs 0
2019-09-03 14:21:33,764 DEV : loss 0.010614411905407906 - score 1.0
2019-09-03 14:21:39,621 TEST : loss 1.0491420030593872 - score 0.752
2019-09-03 14:21:45,625 ----------------------------------------------------------------------------------------------------
2019-09-03 14:21:46,708 epoch 10 - iter 0/7 - loss 0.01868066
2019-09-03 14:21:47,583 epoch 10 - iter 1/7 - loss 0.01256309
2019-09-03 14:21:48,362 epoch 10 - iter 2/7 - loss 0.01022791
2019-09-03 14:21:49,115 epoch 10 - iter 3/7 - loss 0.01004152
2019-09-03 14:21:49,835 epoch 10 - iter 4/7 - loss 0.00964991
2019-09-03 14:21:50,560 epoch 10 - iter 5/7 - loss 0.01175649
2019-09-03 14:21:50,773 epoch 10 - iter 6/7 - loss 0.01078710
2019-09-03 14:21:50,935 ------------------------------------------------------------------------------

2019-09-03 14:24:38,573 ----------------------------------------------------------------------------------------------------
2019-09-03 14:24:40,060 epoch 7 - iter 0/7 - loss 0.18729357
2019-09-03 14:24:40,986 epoch 7 - iter 1/7 - loss 0.16421210
2019-09-03 14:24:41,822 epoch 7 - iter 2/7 - loss 0.15289205
2019-09-03 14:24:42,650 epoch 7 - iter 3/7 - loss 0.13287557
2019-09-03 14:24:43,395 epoch 7 - iter 4/7 - loss 0.15919293
2019-09-03 14:24:44,215 epoch 7 - iter 5/7 - loss 0.16811558
2019-09-03 14:24:44,466 epoch 7 - iter 6/7 - loss 0.15863865
2019-09-03 14:24:44,631 ----------------------------------------------------------------------------------------------------
2019-09-03 14:24:44,632 EPOCH 7 done: loss 0.1586 - lr 0.0010 - bad epochs 0
2019-09-03 14:24:49,170 DEV : loss 0.06640192121267319 - score 0.99
2019-09-03 14:24:54,978 TEST : loss 0.8791607618331909 - score 0.764
2019-09-03 14:25:00,769 -------------------------------------------------------------------------------------

2019-09-03 14:27:31,985 epoch 4 - iter 3/7 - loss 0.61289025
2019-09-03 14:27:32,842 epoch 4 - iter 4/7 - loss 0.59338340
2019-09-03 14:27:33,635 epoch 4 - iter 5/7 - loss 0.64708004
2019-09-03 14:27:33,857 epoch 4 - iter 6/7 - loss 0.72833968
2019-09-03 14:27:34,025 ----------------------------------------------------------------------------------------------------
2019-09-03 14:27:34,026 EPOCH 4 done: loss 0.7283 - lr 0.0010 - bad epochs 0
2019-09-03 14:27:38,706 DEV : loss 0.34064778685569763 - score 0.89
2019-09-03 14:27:44,450 TEST : loss 1.255679965019226 - score 0.66
2019-09-03 14:27:50,599 ----------------------------------------------------------------------------------------------------
2019-09-03 14:27:51,837 epoch 5 - iter 0/7 - loss 0.43405834
2019-09-03 14:27:52,889 epoch 5 - iter 1/7 - loss 0.35755713
2019-09-03 14:27:53,709 epoch 5 - iter 2/7 - loss 0.35195065
2019-09-03 14:27:54,486 epoch 5 - iter 3/7 - loss 0.29316584
2019-09-03 14:27:55,489 epoch 5 - iter 4/7 - loss 

2019-09-03 14:30:30,157 EPOCH 1 done: loss 3.5895 - lr 0.0010 - bad epochs 0
2019-09-03 14:30:34,572 DEV : loss 4.102510929107666 - score 0.275
2019-09-03 14:30:40,135 TEST : loss 4.482346057891846 - score 0.244
2019-09-03 14:30:46,274 ----------------------------------------------------------------------------------------------------
2019-09-03 14:30:47,481 epoch 2 - iter 0/7 - loss 4.44855213
2019-09-03 14:30:48,313 epoch 2 - iter 1/7 - loss 3.45330715
2019-09-03 14:30:49,077 epoch 2 - iter 2/7 - loss 3.23546243
2019-09-03 14:30:49,811 epoch 2 - iter 3/7 - loss 2.69277036
2019-09-03 14:30:50,530 epoch 2 - iter 4/7 - loss 2.38463197
2019-09-03 14:30:51,276 epoch 2 - iter 5/7 - loss 2.15867968
2019-09-03 14:30:51,534 epoch 2 - iter 6/7 - loss 2.15336806
2019-09-03 14:30:51,713 ----------------------------------------------------------------------------------------------------
2019-09-03 14:30:51,714 EPOCH 2 done: loss 2.1534 - lr 0.0010 - bad epochs 0
2019-09-03 14:30:56,342 DEV : loss

...

2019-09-03 22:45:53,691 epoch 8 - iter 12/13 - loss 0.02518275
2019-09-03 22:45:53,987 ----------------------------------------------------------------------------------------------------
2019-09-03 22:45:53,989 EPOCH 8 done: loss 0.0252 - lr 0.0010 - bad epochs 1
2019-09-03 22:46:02,728 DEV : loss 0.020390978083014488 - score 0.9975
2019-09-03 22:46:04,227 TEST : loss 0.14929652214050293 - score 0.94
2019-09-03 22:46:10,940 ----------------------------------------------------------------------------------------------------
2019-09-03 22:46:12,669 epoch 9 - iter 0/13 - loss 0.01481061
2019-09-03 22:46:13,526 epoch 9 - iter 1/13 - loss 0.00996848
2019-09-03 22:46:14,471 epoch 9 - iter 2/13 - loss 0.00757945
2019-09-03 22:46:15,338 epoch 9 - iter 3/13 - loss 0.00886344
2019-09-03 22:46:16,196 epoch 9 - iter 4/13 - loss 0.00869954
2019-09-03 22:46:17,073 epoch 9 - iter 5/13 - loss 0.01019105
2019-09-03 22:46:17,799 epoch 9 - iter 6/13 - loss 0.01377516
2019-09-03 22:46:18,589 epoch 9 - it

2019-09-03 22:48:17,518 EPOCH 3 done: loss 0.3610 - lr 0.0010 - bad epochs 0
2019-09-03 22:48:26,150 DEV : loss 0.26449644565582275 - score 0.9175
2019-09-03 22:48:27,679 TEST : loss 0.47082069516181946 - score 0.86
2019-09-03 22:48:34,674 ----------------------------------------------------------------------------------------------------
2019-09-03 22:48:36,367 epoch 4 - iter 0/13 - loss 0.40814841
2019-09-03 22:48:37,220 epoch 4 - iter 1/13 - loss 0.37758704
2019-09-03 22:48:37,957 epoch 4 - iter 2/13 - loss 0.32885639
2019-09-03 22:48:38,786 epoch 4 - iter 3/13 - loss 0.31114438
2019-09-03 22:48:39,864 epoch 4 - iter 4/13 - loss 0.27022263
2019-09-03 22:48:40,637 epoch 4 - iter 5/13 - loss 0.24020104
2019-09-03 22:48:41,371 epoch 4 - iter 6/13 - loss 0.22586164
2019-09-03 22:48:42,194 epoch 4 - iter 7/13 - loss 0.22774020
2019-09-03 22:48:43,059 epoch 4 - iter 8/13 - loss 0.22420446
2019-09-03 22:48:43,913 epoch 4 - iter 9/13 - loss 0.21073208
2019-09-03 22:48:44,784 epoch 4 - iter 

2019-09-03 22:51:11,009 epoch 10 - iter 4/13 - loss 0.01699753
2019-09-03 22:51:11,896 epoch 10 - iter 5/13 - loss 0.01492753
2019-09-03 22:51:12,627 epoch 10 - iter 6/13 - loss 0.02059848
2019-09-03 22:51:13,366 epoch 10 - iter 7/13 - loss 0.01858976
2019-09-03 22:51:14,257 epoch 10 - iter 8/13 - loss 0.01771024
2019-09-03 22:51:15,076 epoch 10 - iter 9/13 - loss 0.02062650
2019-09-03 22:51:15,812 epoch 10 - iter 10/13 - loss 0.02093051
2019-09-03 22:51:16,591 epoch 10 - iter 11/13 - loss 0.02314158
2019-09-03 22:51:17,122 epoch 10 - iter 12/13 - loss 0.02421540
2019-09-03 22:51:17,445 ----------------------------------------------------------------------------------------------------
2019-09-03 22:51:17,448 EPOCH 10 done: loss 0.0242 - lr 0.0010 - bad epochs 0
2019-09-03 22:51:26,078 DEV : loss 0.016539692878723145 - score 1.0
2019-09-03 22:51:27,622 TEST : loss 0.15429499745368958 - score 0.94
2019-09-03 22:51:41,890 ------------------------------------------------------------------

2019-09-03 22:53:41,504 epoch 5 - iter 7/13 - loss 0.20726207
2019-09-03 22:53:42,297 epoch 5 - iter 8/13 - loss 0.19749610
2019-09-03 22:53:43,096 epoch 5 - iter 9/13 - loss 0.19240389
2019-09-03 22:53:43,901 epoch 5 - iter 10/13 - loss 0.18399881
2019-09-03 22:53:44,634 epoch 5 - iter 11/13 - loss 0.18378617
2019-09-03 22:53:45,221 epoch 5 - iter 12/13 - loss 0.17573844
2019-09-03 22:53:45,545 ----------------------------------------------------------------------------------------------------
2019-09-03 22:53:45,548 EPOCH 5 done: loss 0.1757 - lr 0.0010 - bad epochs 0
2019-09-03 22:53:54,731 DEV : loss 0.13796302676200867 - score 0.97
2019-09-03 22:53:56,379 TEST : loss 0.23280271887779236 - score 0.92
2019-09-03 22:54:03,638 ----------------------------------------------------------------------------------------------------
2019-09-03 22:54:05,232 epoch 6 - iter 0/13 - loss 0.09524014
2019-09-03 22:54:06,173 epoch 6 - iter 1/13 - loss 0.13761004
2019-09-03 22:54:06,959 epoch 6 - ite

2019-09-03 22:56:24,807 ----------------------------------------------------------------------------------------------------


#### SVM + <font color=#007700>TF-IDF</font>

In [152]:
for language in ['en', 'es', 'it', 'nl']:
    print('\n\nLanguage: ', language)
    dataset = load_disequa(language)
    create_feature('tfidf', dataset, dataset, max_features=2000)
    
    model = {'name': 'svm', 'model': svm_linear}
    
    tfidf = np.array([list(r) for r in dataset['tfidf'].values])
    tfidf = normalize(tfidf, norm='max')
    
    X = np.array([list(x) for x in dataset['tfidf'].values])
    y = dataset['class'].values
    
    run_benchmark_cv(model, X, y, sizes_train=[100,200,300,400],
                     save='results/DISEQuA_svm_tfidf_' + language + '.csv')



Language:  en

100|..........
200|..........
300|..........
400|..........
Run time benchmark: 1.027012586593628


Language:  es

100|..........
200|..........
300|..........
400|..........
Run time benchmark: 1.0114972591400146


Language:  it

100|..........
200|..........
300|..........
400|..........
Run time benchmark: 1.1434721946716309


Language:  nl

100|..........
200|..........
300|..........
400|..........
Run time benchmark: 1.1250619888305664


#### SVM + <font color=#007700>TF-IDF</font> + <font color=#0055CC>WB</font>

In [163]:
for language in ['en', 'es', 'it', 'nl']:
    print('\n\nLanguage: ', language)
    embedding = load_embedding(path_wordembedding + 'wiki.multi.' + language + '.vec')
    dataset = load_disequa(language)
    create_feature('tfidf', dataset, dataset, max_features=2000)
    create_feature('embedding_sum', None, dataset, embedding)
    
    model = {'name': 'svm', 'model': svm_linear}
    
    tfidf = np.array([list(r) for r in dataset['tfidf'].values])
    tfidf = normalize(tfidf, norm='max')
    
    embedding = np.array([list(r) for r in dataset['embedding_sum'].values])
    embedding = normalize(embedding, norm='max')
    
    X = np.array([list(x) + list(xx) for x, xx in zip(tfidf, embedding)])
    y = dataset['class'].values
    
    run_benchmark_cv(model, X, y, sizes_train=[100,200,300,400],
                     save='results/DISEQuA_svm_cortes_' + language + '.csv')



Language:  en

100|..........
200|..........
300|..........
400|..........
Run time benchmark: 6.358882427215576


Language:  es

100|..........
200|..........
300|..........
400|..........
Run time benchmark: 7.197380065917969


Language:  it

100|..........
200|..........
300|..........
400|..........
Run time benchmark: 5.5334153175354


Language:  nl

100|..........
200|..........
300|..........
400|..........
Run time benchmark: 6.624628782272339


#### SVM + <font color=#007700>TF-IDF</font> + <font color=#0055CC>WB</font> + <font color=#CC6600>POS</font> + <font color=#CC6600>NER</font>

In [164]:


for language in ['en', 'es', 'it', 'nl']:
    print('\n\nLanguage: ', language)
    embedding = load_embedding(path_wordembedding + 'wiki.multi.' + language + '.vec')
    dataset = load_disequa(language)
    create_feature('tfidf', dataset, dataset, max_features=2000)
    create_feature('embedding_sum', dataset, dataset, embedding)
    create_feature('pos_hotencode', dataset, dataset)
    create_feature('ner_hotencode', dataset, dataset)
    model = {'name': 'svm', 'model': svm_linear}
    
    tfidf = np.array([list(r) for r in dataset['tfidf'].values])
    tfidf = normalize(tfidf, norm='max')
    
    embedding = np.array([list(r) for r in dataset['embedding_sum'].values])
    embedding = normalize(embedding, norm='max')
    
    pos = np.array([list(r) for r in dataset['pos_hotencode'].values])
    
    ner = np.array([list(r) for r in dataset['ner_hotencode'].values])
    
    X = np.array([list(x) + list(xx) + list(xxx) + list(xxxx) for x, xx, xxx, xxxx in zip(tfidf, embedding, pos, ner)])
    
    y = dataset['class'].values
    
    run_benchmark_cv(model, X, y, sizes_train=[100,200,300,400],
                     save='results/DISEQuA_svm_high_' + language + '.csv')



Language:  en

100|..........
200|..........
300|..........
400|..........
Run time benchmark: 6.811999559402466


Language:  es

100|..........
200|..........
300|..........
400|..........
Run time benchmark: 8.384974479675293


Language:  it

100|..........
200|..........
300|..........
400|..........
Run time benchmark: 6.426969528198242


Language:  nl

100|..........
200|..........
300|..........
400|..........
Run time benchmark: 6.852076053619385


## Old stuffs bellow

#### CNN

In [ ]:
# 'en', 'es'
for language in ['es']:
    print('\n\nLanguage: ', language)
    #embedding = load_embedding(path_wordembedding + 'wiki.multi.' + language + '.vec')
    dataset_train, dataset_test = load_uiuc(language)
    text_representation = 'vocab_index'
    vocabulary_inv = create_feature(text_representation, dataset_train, dataset_train)
    create_feature(text_representation, dataset_train, dataset_test)
    model = {'name': 'cnn', 'model': cnn}
    X_train = np.array([list(x) for x in dataset_train[text_representation].values])
    X_test = np.array([list(x) for x in dataset_test[text_representation].values])
    #X_train = pad_sequences(X_train, maxlen=12, dtype='float', padding='post', truncating='post', value=0.0)
    #X_test = pad_sequences(X_test, maxlen=12, dtype='float', padding='post', truncating='post', value=0.0)
    y_train = dataset_train['class'].values
    y_test = dataset_test['class'].values
    ohe = OneHotEncoder()
    y_train = ohe.fit_transform([[y_] for y_ in y_train]).toarray()
    y_test = ohe.transform([[y_] for y_ in y_test]).toarray()
    # , 
    run_benchmark(model, X_train, y_train, X_test, y_test, sizes_train=[1000, 2000, 3000, 4000, 5500],
                  runs=30, save='results/UIUC_cnn_' + language + '.csv', epochs=100, onehot=ohe,
                  vocabulary_size=len(vocabulary_inv))

#### LSTM + WordEmbedding

In [73]:
for language in ['es']:
    print('\n\nLanguage: ', language)
    embedding = load_embedding(path_wordembedding + 'wiki.multi.' + language + '.vec')
    dataset_train, dataset_test = load_uiuc(language)
    dataset_train = dataset_train[:100]
    #dataset_test = dataset_test[:10]
    create_feature('embedding', dataset_train, dataset_train, embedding)
    create_feature('embedding', dataset_train, dataset_test, embedding)
    model = {'name': 'lstm', 'model': lstm_default}
    #print(dataset_train['embedding'].values.shape)
    #print(dataset_train['embedding'].values.dtype)
    #print(dataset_test['embedding'].values.shape)
    X_train = np.array([list(x) for x in dataset_train['embedding'].values])
    X_test = np.array([list(x) for x in dataset_test['embedding'].values])
    X_train = pad_sequences(X_train, maxlen=12, dtype='float', padding='post', truncating='post', value=0.0)
    X_test = pad_sequences(X_test, maxlen=12, dtype='float', padding='post', truncating='post', value=0.0)
    y_train = dataset_train['class'].values
    y_test = dataset_test['class'].values
#     y_train_sub = dataset_train['sub_class'].values
#     sub_classes = set()
#     for sc in y_train_sub:
#         sub_classes.add(sc)
#     y_test_sub = dataset_test['sub_class'].values
#     X_test_sub_ = []
#     y_test_sub_ = []
#     for i in range(len(X_test)):
#         if y_train_sub[i] in sub_classes:
#             X_test_sub_.append(X_test[i])
#             y_test_sub_.append(y_train_sub[i])
#     X_test_sub_ = np.array(X_test_sub_)
#     y_test_sub_ = np.array(y_test_sub_)
    ohe = OneHotEncoder()
    y_train = ohe.fit_transform([[y_] for y_ in y_train]).toarray()
    y_test = ohe.transform([[y_] for y_ in y_test]).toarray() 
    run_benchmark(model, X_train, y_train, X_test, y_test, sizes_train=[1000, 2000, 3000, 4000, 5500],
                  runs=30, save='results/UIUC_lstm_embedding_' + language + '_2.csv', epochs=100, onehot=ohe)
    #run_benchmark(model, X_train, y_train_sub, X_test_sub_, y_test_sub_, sizes_train=[1000, 2000, 3000, 4000, 5500],
    #              save='results/UIUCsub_svm_tfidf_' + language + '.csv')



Language:  es
(100,)
object
(1349,)

1000|...
2000|...
3000|...
4000|...
5500|...
Run time benchmark: 228.79835891723633


#### LSTM + BERT

In [ ]:
for language in ['en']:
    print('\n\nLanguage: ', language)
    #embedding = load_embedding(path_wordembedding + 'wiki.multi.' + language + '.vec')
    dataset_train, dataset_test = load_uiuc(language)
    # debug
    print('WARNING: use subset (first 1000 entries) of training data')
    #dataset_train = dataset_train[:5500].copy()
    
    create_feature('bert', dataset_train, dataset_train)
    create_feature('bert', dataset_train, dataset_test)
    model = {'name': 'lstm', 'model': lstm_default}
    X_train = dataset_train['bert'].values
    X_test = dataset_test['bert'].values
    
    X_train = np.array([x for x in X_train])
    X_test = np.array([x for x in X_test])
    
    #X_train = pad_sequences(X_train, maxlen=12, dtype='float', padding='post', truncating='post', value=0.0)
    #X_test = pad_sequences(X_test, maxlen=12, dtype='float', padding='post', truncating='post', value=0.0)
    y_train = dataset_train['class'].values
    y_test = dataset_test['class'].values
    ohe = OneHotEncoder()
    y_train = ohe.fit_transform([[y_] for y_ in y_train]).toarray()
    y_test = ohe.transform([[y_] for y_ in y_test]).toarray() 
    run_benchmark(model, X_train, y_train, X_test, y_test, sizes_train=[5500], # 1000, 2000, 3000, 4000, 5500
                  runs=1, save='results/UIUC_lstm_bert_' + language + '.csv', 
                  epochs=100, onehot=ohe, in_dim=768)
    #run_benchmark(model, X_train, y_train_sub, X_test_sub_, y_test_sub_, sizes_train=[1000, 2000, 3000, 4000, 5500],
    #              save='results/UIUCsub_svm_tfidf_' + language + '.csv')

## DISEQuA Benchmark

### RUN DISEQuA Benchmark

##### SVM + TFIDF

In [ ]:
for language in ['DUT', 'ENG', 'ITA', 'SPA']:
    print('\n\nLanguage: ', language)
    dataset = load_disequa(language)
    create_feature('tfidf', dataset, dataset, embedding)
    model = {'name': 'svm', 'model': svm_linear}
    X = np.array([list(x) for x in dataset['tfidf'].values])
    y = dataset['class'].values
    run_benchmark(model, X, y, sizes_train=[100,200,300,400,405],
                  save='results/DISEQuA_svm_tfidf_' + language + '.csv')

##### RFC + TFIDF

In [ ]:
for language in ['DUT', 'ENG', 'ITA', 'SPA']:
    print('\n\nLanguage: ', language)
    dataset = load_disequa(language)
    create_feature('tfidf', dataset, dataset, embedding)
    model = {'name': 'rfc', 'model': random_forest}
    X = np.array([list(x) for x in dataset['tfidf'].values])
    y = dataset['class'].values
    run_benchmark(model, X, y, sizes_train=[100,200,300,400],
                  save='results/DISEQuA_rfc_tfidf_' + language + '.csv')

##### SVM + TFIDF_3gram + SKB

In [ ]:
for language in ['DUT', 'ENG', 'ITA', 'SPA']:
    print('\n\nLanguage: ', language)
    dataset = load_disequa(language)
    create_feature('tfidf_3gram', dataset, dataset)
    model = {'name': 'svm', 'model': svm_linear}
    X = np.array([list(x) for x in dataset['tfidf'].values])
    y = dataset['class'].values
    skb = SelectKBest(chi2, k=2000).fit(X, y)
    X = skb.transform(X)
    run_benchmark(model, X, y, sizes_train=[100,200,300,400],
                  save='results/DISEQuA_svm_tfidf_3gram_' + language + '.csv')

##### LSTM + Embedding

In [ ]:
for language, embd_l in zip(['SPA'], ['es']):
    print('\n\nLanguage: ', language)
    embedding = load_embedding(path_wordembedding + 'wiki.multi.' + embd_l + '.vec')
    dataset = load_disequa(language)
    create_feature('embedding', dataset, dataset, embedding)
    model = {'name': 'lstm', 'model': lstm_default}
    X = np.array([list(x) for x in dataset['embedding'].values])
    y = dataset['class'].values
    X = pad_sequences(X, maxlen=12, dtype='float', padding='post', truncating='post', value=0.0)
    ohe = OneHotEncoder()
    y = ohe.fit_transform([[y_] for y_ in y]).toarray()
    run_benchmark(model, X, y, sizes_train=[100,200,300,400,405], onehot=ohe,
                  save='results/DISEQuA_lstm_embedding_' + language + '.csv')

##### CNN

In [ ]:
for language, embd_l in zip(['DUT', 'ENG', 'ITA', 'SPA'], ['nl', 'eng', 'it', 'es']):
    print('\n\nLanguage: ', language)
    #embedding = load_embedding(path_wordembedding + 'wiki.multi.' + embd_l + '.vec')
    dataset = load_disequa(language)
    text_representation = 'vocab_index'
    vocabulary_inv = create_feature(text_representation, dataset, dataset)
    model = {'name': 'cnn', 'model': cnn}
    X = np.array([list(x) for x in dataset[text_representation].values])
    y = dataset['class'].values
    #X = pad_sequences(X, maxlen=12, dtype='float', padding='post', truncating='post', value=0.0)
    ohe = OneHotEncoder()
    y = ohe.fit_transform([[y_] for y_ in y]).toarray()
    run_benchmark(model, X, y, sizes_train=[100,200,300,400], onehot=ohe, vocabulary_size=len(vocabulary_inv),
                  save='results/DISEQuA_cnn_' + language + '.csv', epochs=100)